#### 스타벅스 서울시 매장위치 크롤링

##### 스타벅스 웹사이트
https://www.starbucks.co.kr/index.do

##### 셀레니움 등 라이브러리 임포트

In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.autonotebook import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [13]:
# 크롬 웹 드라이버 실행
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [14]:
# 웹드라이버 선택할 위치값 작성
btn_seoul = '#container > div > form > fieldset > div > section > article.find_store_cont > ' + \
            'article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > ' + \
            'li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, btn_seoul).click()

In [15]:
# 서울이 선택된 상태
# '전체' 를 클릭
btn_all = '#mCSB_2_container > ul > li > a' # 첫 번째 값이기 때문에 nth-child 필요 X
# btn_all
driver.find_element(By.CSS_SELECTOR, btn_all).click()

In [17]:
# 스타벅스는 한 화면에 전부 리스트업 되기 때문에 함수를 만들 필요도 없음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [19]:
# 검색결과 위치
starbucks_soup_list = soup.select('li.quickResultLstCon')
len(starbucks_soup_list)

600

In [20]:
starbucks_soup_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [45]:
# 매장정보 확인
starbucks_store = starbucks_soup_list[0]
name = starbucks_store.select('strong')[0].text.strip()
lat = starbucks_store['data-lat'].strip()
lng = starbucks_store['data-long'].strip()
store_type = starbucks_store.select('i')[0]['class'][0].split('_')[1] # _로 자르면 두번째 값이 general / reserve
address = starbucks_store.select('p')[0].get_text(separator='|').split('|')[0]


print(name) # 매장명
print(lat, lng, sep= ' / ') # 매장 지도 위치
print(store_type)
print(address)

역삼아레나빌딩
37.501087 / 127.043069
general
서울특별시 강남구 언주로 425 (역삼동)


In [47]:
# 600개 매장 데이터 가져오기
starbucks_seoul_list = []

for item in tqdm(starbucks_soup_list):
    name = starbucks_store.select('strong')[0].text.strip()
    lat = starbucks_store['data-lat'].strip()
    lng = starbucks_store['data-long'].strip()
    store_type = starbucks_store.select('i')[0]['class'][0].split('_')[1] # _로 자르면 두번째 값이 general / reserve
    address = starbucks_store.select('p')[0].get_text(separator='|').split('|')[0]

    starbucks_seoul_list.append(name, lat, lng, store_type, address)

NameError: name 'tqdm' is not defined

In [ ]:
starbucks_seoul_list

In [ ]:
# DataFrame 생성
columns = ['매장명', '위도', '경도', '매장타입', '주소']
df = pd.DataFrame(starbucks_seoul_list, columns=columns)
df.tail()

In [ ]:
# 엑셀로 저장
df.to_excel('./seoul_starbucks.xlsx')